In [1]:
pip install num2words

     |████████████████████████████████| 102kB 3.8MB/s 


In [2]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat May  9 22:34:44 2020

@author: sakib

"""


#change base directory
import os

base_path=r'/content'
working_directory = '/content'
data_folder = r'/content/data'
embedding_folder = r'/content/embeddings'
models_folder = r'/content/models'
model_directory = working_directory+'/'+'models'


def change_base_dir(base_dir_path):
    """ Change the working directopry of the code"""
    
    if not os.path.exists(base_dir_path):
        print ('creating directory', base_dir_path)
        os.makedirs(base_dir_path)
    print ('Changing base directory to ', base_dir_path)
    os.chdir(base_dir_path)
 
      

# base_folder='data'
# base_dir_path=base_path+'/'+base_folder
# change_base_dir(base_dir_path)
# download_data.extract_file('/media/sakib/alpha/work/EmotionDetectionDir/git/data/glove.6B.zip')



change_base_dir(base_path)
if not os.path.exists(data_folder):
        os.makedirs(data_folder)
if not os.path.exists(embedding_folder):
        os.makedirs(embedding_folder)
if not os.path.exists(models_folder):
        os.makedirs(models_folder)
  
#loading necessary files
from load_preprocess import load_data,load_data_embedding
from load_preprocess import preprocess_data
from load_preprocess import read_labels
import word2vec
import sswe
import glove_file
import designing_network
import download_data


#importing libraries
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import pandas as pd



# Download data

print('Checking requirements:')
download_data.download_data(base_path = base_path)

#loading data
print('Step1: Loading Embedding training Dataset...')


dataset_embedding = load_data_embedding(working_directory+'/data/'+'training.1600000.processed.noemoticon.csv')
print('Step2: Shuffling data...')
dataset_embedding = dataset_embedding.sample(frac=1) # reshuffling the data
print('Step3: Preprocessing the texts...')
texts = preprocess_data(dataset_embedding)
labels = read_labels(dataset_embedding)


# building word2vec model
print('Step4: Building word2vec model...')
EMBEDDING_DIM = 100
w2v = word2vec.create_word2vec(directory = working_directory+'/'+'embeddings',texts = texts ,min_count = 1,EMBEDDING_DIM = EMBEDDING_DIM)




# loading twitter_dataset_small
print('Step6: Loading Twitter Dataset')
dataset = load_data(working_directory+'/'+'data'+'/'+'Tweets.csv')
texts = preprocess_data(dataset)
y = pd.get_dummies(dataset['Label']).values



#tokenizing
print ('Step7: Tokenizing...')

tokens = []
for line in texts:
    words = word_tokenize(line)
    tokens.append(words)



tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(tokens)
sequences = tokenizer_obj.texts_to_sequences(tokens)


#padding
print ('Step8: Padding...')
tokenizer_word_index = tokenizer_obj.word_index
max_length = 150
review_pad = pad_sequences(sequences, maxlen = max_length)



# train test split
print('Step9: train and test set generation...')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review_pad, y, test_size = 0.20, random_state = 0)

num_words = len(tokenizer_word_index) + 1

# sswe embedding matrix
print('Step12: Generating sswe embedding matrix...')
sswe_embedding_filename = working_directory+'/'+'embeddings'+'/'+'embeddings_sswe.tsv'
embedding_matrix_sswe = sswe.load_sswe(filename = sswe_embedding_filename, tokenizer_word_index = tokenizer_word_index, EMBEDDING_DIM = 50)



Changing base directory to  /content


Using TensorFlow backend.


Checking requirements:
Extracting stanford data
Extracting glove data
Step1: Loading Embedding training Dataset...
Step2: Shuffling data...
Step3: Preprocessing the texts...
Step4: Building word2vec model...
Saving word2vec model in the disk


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Step6: Loading Twitter Dataset
Step7: Tokenizing...
Step8: Padding...
Step9: train and test set generation...
Step12: Generating sswe embedding matrix...


FileNotFoundError: ignored

In [9]:



# Download data

print('Checking requirements:')
download_data.download_data(base_path = base_path)

#loading data
print('Step1: Loading Embedding training Dataset...')


dataset_embedding = load_data_embedding(working_directory+'/data/'+'training.1600000.processed.noemoticon.csv')
print('Step2: Shuffling data...')
dataset_embedding = dataset_embedding.sample(frac=1) # reshuffling the data
print('Step3: Preprocessing the texts...')
texts = preprocess_data(dataset_embedding)
labels = read_labels(dataset_embedding)


# building word2vec model
print('Step4: Building word2vec model...')
EMBEDDING_DIM = 100

#building sswe model
print('Step5: Building sswe model...')
sswe_model,training_word_index = sswe.sswe_model(texts, labels)
embedding_weights, word_indices_df, merged = sswe.save_sswe(sswe_model,training_word_index,directory = working_directory+'/'+'embeddings')
print('Embedding Layers are trained.')


# loading twitter_dataset_small
print('Step6: Loading Twitter Dataset')
dataset = load_data(working_directory+'/'+'data'+'/'+'Tweets.csv')
texts = preprocess_data(dataset)
y = pd.get_dummies(dataset['Label']).values



#tokenizing
print ('Step7: Tokenizing...')

tokens = []
for line in texts:
    words = word_tokenize(line)
    tokens.append(words)



tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(tokens)
sequences = tokenizer_obj.texts_to_sequences(tokens)


#padding
print ('Step8: Padding...')
tokenizer_word_index = tokenizer_obj.word_index
max_length = 150
review_pad = pad_sequences(sequences, maxlen = max_length)



# train test split
print('Step9: train and test set generation...')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review_pad, y, test_size = 0.20, random_state = 0)





# sswe embedding matrix
print('Step12: Generating sswe embedding matrix...')
sswe_embedding_filename = working_directory+'/'+'embeddings'+'/'+'embeddings_sswe.tsv'
embedding_matrix_sswe = sswe.load_sswe(filename = sswe_embedding_filename, tokenizer_word_index = tokenizer_word_index, EMBEDDING_DIM = 50)



Checking requirements:
Stanford dataset already downloaded...
Twitter dataset already downloaded...
Glove model already downloaded...
Step1: Loading Embedding training Dataset...
Step2: Shuffling data...
Step3: Preprocessing the texts...
Step4: Building word2vec model...
Step5: Building sswe model...
1600000 1600000
Using Keras tokenizer to tokenize and build word index
Size of the vocab is 352268
Padding sentences and shuffling the sswe train data
Initializing the model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 50)            17613500  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 15)            3765      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 15)             0         
________________________________________

/content/sswe.py:129: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(train_x, train_y,nb_epoch=no_epochs, batch_size=batch_size,validation_data=(valid_x, valid_y),callbacks=[mcp])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1280000 samples, validate on 320000 samples
Epoch 1/5
1280000/1280000 [==============================] - 69s 54us/step - loss: 0.5384 - acc: 0.7412 - val_loss: 0.5069 - val_acc: 0.7598
Epoch 2/5
1280000/1280000 [==============================] - 69s 54us/step - loss: 0.5133 - acc: 0.7513 - val_loss: 0.5293 - val_acc: 0.7477
Epoch 3/5
1280000/1280000 [==============================] - 69s 54us/step - loss: 0.5291 - acc: 0.7452 - val_loss: 0.5341 - val_acc: 0.7426
Epoch 4/5
1280000/1280000 [==============================] - 69s 54us/step - loss: 0.5436 - acc: 0.7368 - val_loss: 0.5688 - val_acc: 0.7316
Epoch 5/5
1280000/1280000 [==============================] - 68s 53us/step - loss: 0.5645 - acc: 0.7217 - val_loss: 0.5629 - val_acc: 0.7304
(352268, 2) (352270, 51)
(352268, 52)
Embedding Layers are trained.
Step6: Loading Twitter Dataset
Step7: Tokenizing...
Step8: Padding...
Step9: train and test set generation...
Step12: Generating sswe embedding matrix...
Step13: designing ls

NameError: ignored

In [10]:


import designing_networkss
# training the sswe model with lstm
print('Step13: designing lstm+sswe model...')
sswe_lstm = designing_networkss.model_architecture_sswe(embedding_matrix_sswe, num_words,EMBEDDING_DIM = 50 , max_length = max_length)
sswe_lstm, history = designing_networkss.fit_network(sswe_lstm, X_train, X_test, y_train, y_test)
designing_networkss.save_network_model(sswe_lstm, modelname = 'sswe_lstm',directory = model_directory)
# loaded_model = designing_network.load_network_model( directory = working_directory+'/'+'models', jsonfile = 'sswe_lstm.json', h5file = 'sswe_lstm.h5')
# score = designing_network.analyze_performance(loaded_model, X_test, y_test)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


Step13: designing lstm+sswe model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 50)           691150    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                21248     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 712,593
Trainable params: 21,443
Non-trainable params: 691,150
_________________________________________________________________
Train on 11712 samples, validate on 2928 samples
Epoch 1/10
11712/11712 [==============================] - 40s 3ms/step - loss: 0.4876 - accuracy: 0.7803 - val_loss: 0.4350 - val_accuracy: 0.8067
Epoch 2/10
11712/11712 [==============================] - 40s 3ms/step - loss: 0.4309 - accuracy: 0.8092 - val_loss: 0.4053 - val_accuracy: 